## Thermodynamics of the dimerization of formic acid  

### Demian Riccardi
#### February 22, 2016

The purpose of this notebook is to analyze the thermodynamics of the dimerization of formic acid.

Resource: Exercise 7.2.1 from Tom Record's Biophysical Chemistry textbook handouts. 

Language: Perl

CPAN Modules: Math::Polynomial::Solve qw(poly_roots)

In [1]:
use Modern::Perl;
use Math::Polynomial::Solve qw(poly_roots);
IPerl->load_plugin( "ChartClicker" );

### Quadratic equation
Equilibria problems often a second-order polynomial, which may be solved using the quadratic formula.  The quadratic formula provides solutions for x and should be familiar for most college students, 

### $a x^2 + b x + c = 0  $

### $ (x_-,x_+) = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$.

Where the to two solutions $(x_-,x_+)$ correspond to using the - and + forms of the equation, respectively.  All the time, I think, only one of the roots makes sense with respect to the parameters of the problem (e.g. "there are no negative concentrations" or "that root is too high").  


### We need to solve for the value of the progress variable at equilibrium
For $2M \rightarrow D$, we have derived that 

free energy of the system:

$G_{sys} = n_m\mu_m + n_d\mu_d $

progress variable:
$\lambda = n_d$

$2n_d + n_m = 2$

$n_m = 2-2\lambda = 2(1-\lambda)$

### For an Ideal Gas, we can write
$\Delta G_{sys} = [\mu_D^{\bullet}+RT \ln \left(\frac{\lambda}{2-\lambda} \right)] - 2 [\mu_M^{\bullet}+ RT \ln \left(\frac{2-2\lambda}{2-\lambda} \right) ]$ 

At equilibrium, $\Delta G_{sys} = 0$, and with a little algebra and log magic you should arrive at
$0 = (4+e)\lambda^2 - (8+2e)\lambda+4$ where e = exp(-3.36/RT)

In [2]:
my $NA  = 6.022E23;
my $T   = 298 ; #kelvin
my $R   =  8.3144598 ; #J 
my $Rkcal = ($R/4.184) * (1/1000) ; # cal/mol.K and then kcal/mol
my $mu_Mp = 1.68  ; # chemical potential of pure monomer, defined relative to D
my $mu_Dp = 0     ; # chemical potential of pure dimer
my $RT = $Rkcal*$T;

my $e = exp(-3.36/($RT)) ;
my $a = 4+$e;
my $b = -8-2*$e;
my $c = 4;
my @roots = poly_roots($a,$b,$c);
say foreach @roots;


1.0292896928186
0.970710307181397


### since $\lambda$ goes from 0 to 1, $\lambda_{eq}= 0.97$

### Let's do a little plotting.  Try to understand the code here.  

A. Set explicit values of lambda from 0 to 1.


In [3]:
my $max_points = 20;
my @lambdas = map {$_/($max_points)} 0 .. $max_points;

#add a little to avoid the log of zero

$lambdas[0] = $lambdas[0] + 0.0001;
$lambdas[$max_points] = $lambdas[$max_points] - 0.0001;

printf("%10.2f\n", $lambdas[$_]) foreach 0 .. $max_points; 

      0.00
      0.05
      0.10
      0.15
      0.20
      0.25
      0.30
      0.35
      0.40
      0.45
      0.50
      0.55
      0.60
      0.65
      0.70
      0.75
      0.80
      0.85
      0.90
      0.95
      1.00


B. Calculate $G_{unmixed}^{\bullet}$ foreach of the lambdas
 
 $G_{unmixed} = n_M \mu_M^{\bullet} + n_d \mu_D^{\bullet} = (2-\lambda)\mu_M^{\bullet} + \lambda \mu_D^{\bullet}$

In [4]:
sub G_unmixed {
    my $lambda = shift;
    return ((2-2*$lambda)*1.68 + $lambda * 0)
}

my @G_unmixed = map{ G_unmixed($_) } @lambdas;
printf("%10.2f %10.2f\n", $lambdas[$_], $G_unmixed[$_]) foreach 0 .. $#lambdas; 

      0.00       3.36
      0.05       3.19
      0.10       3.02
      0.15       2.86
      0.20       2.69
      0.25       2.52
      0.30       2.35
      0.35       2.18
      0.40       2.02
      0.45       1.85
      0.50       1.68
      0.55       1.51
      0.60       1.34
      0.65       1.18
      0.70       1.01
      0.75       0.84
      0.80       0.67
      0.85       0.50
      0.90       0.34
      0.95       0.17
      1.00       0.00


  C. Calculate the $\Delta G_{mixing}$ foreach value of the lambdas
  
  $\Delta G_{mixing} = RT (2-2\lambda)\ln\frac{2-2\lambda}{2-\lambda} + RT \lambda \ln \frac{\lambda}{2-\lambda}$

In [5]:
sub DeltaG_mixing {
    my $lambda = shift;
    my $RT = shift;
    return ($RT * (2-2*$lambda)* (log (2-2*$lambda) - log(2-$lambda)) + $RT * $lambda * (log($_)- log(2-$lambda)) )
}

my @DeltaG_mixing = map { DeltaG_mixing($_, $RT) } @lambdas; 
 
printf("%10.2f %10.2f\n", $lambdas[$_], $DeltaG_mixing[$_]) foreach 0 .. $#lambdas; 

      0.00      -0.00
      0.05      -0.14
      0.10      -0.23
      0.15      -0.31
      0.20      -0.37
      0.25      -0.43
      0.30      -0.47
      0.35      -0.50
      0.40      -0.53
      0.45      -0.55
      0.50      -0.57
      0.55      -0.57
      0.60      -0.57
      0.65      -0.55
      0.70      -0.53
      0.75      -0.50
      0.80      -0.45
      0.85      -0.39
      0.90      -0.31
      0.95      -0.20
      1.00      -0.00


### Finally, calculate $G_{system} = G^{\bullet}_{unmixed} + \Delta G_{mixing}$

In [7]:
my @G_sys = map {DeltaG_mixing($_,$RT) + G_unmixed($_)} @lambdas;

printf("%10.2f %10.2f\n", $lambdas[$_], $G_sys[$_]) foreach 0 .. $#lambdas; 

      0.00       3.36
      0.05       3.05
      0.10       2.79
      0.15       2.55
      0.20       2.32
      0.25       2.09
      0.30       1.88
      0.35       1.68
      0.40       1.48
      0.45       1.30
      0.50       1.11
      0.55       0.94
      0.60       0.78
      0.65       0.62
      0.70       0.48
      0.75       0.34
      0.80       0.22
      0.85       0.11
      0.90       0.03
      0.95      -0.03
      1.00      -0.00


#### Feed the data into Chart::Clicker

In [8]:
my $Gunmixed = Chart::Clicker::Data::Series->new(
    keys    => \@lambdas,
    values  => \@G_unmixed,
);

my $DeltaG_mixing = Chart::Clicker::Data::Series->new(
    label   => "DG_mixing",
    keys    => \@lambdas,
    values  => \@DeltaG_mixing,
);

my $G_sys = Chart::Clicker::Data::Series->new(
    keys    => \@lambdas,
    values  => \@G_sys,
);
1;

Chart::Clicker::Data::Series=HASH(0x2705608)


In [9]:
my $cc = Chart::Clicker->new;

my $def = $cc->get_context('default');
$def->domain_axis->fudge_amount(.05);
$def->range_axis->fudge_amount(.01);

my $ds = Chart::Clicker::Data::DataSet->new(series => [ $Gunmixed, $DeltaG_mixing, $G_sys]);

$cc->add_to_datasets($ds);

$cc;